In [ ]:
from collections import deque, defaultdict, OrderedDict
import re
import random
from helper import aoc_timer

In [ ]:
@aoc_timer
def get_input(path):
    return [x.strip() for x in open(path).readlines() if x != '\n']

def get_reactions(data, reverse=False):
    if not reverse:
        # Parse reactions into dictionary {IN: [*OUT]}
        R = defaultdict(list)
        for react in data:
            IN, OUT = react.split(' => ')
            R[IN].append(OUT)
    else:
        # Reactions in reverse
        R = {}
        for react in data:
            OUT, IN = react.split(' => ')
            R[IN] = OUT
        # Sort by length of RHS molecule
        R = OrderedDict(sorted(R.items(), key=lambda x: len(x[0]), reverse=True))
    return R

def shuffle_dict(d):
    items = list(d.items())
    random.shuffle(items)
    return OrderedDict(items)

def repl_all(s, mol, repl):
    """Replace each occurrence of mol in s with repl.
       Return set of all possible single replacements.
    """
    inds = [m.start() for m in re.finditer(f'(?={mol})', s)]
    if not inds:
        return {}
    out = set()
    chars = len(mol)
    for idx in inds:
        pre, post = s[:idx], s[idx+chars:]
        out.add(pre + repl + post)
    return out

def repl_greedy(s, mol, repl):
    """Replace all occurrences of mol in s with repl.
       Return reduced string and count of replacements.
    """
    cnt = 0
    while mol in s:
        s = s.replace(mol, repl, 1)
        cnt += 1
    return s, cnt

In [ ]:
@aoc_timer
def part1(data):
    # Remove medicine from input data into new variable
    medicine = data.pop()

    # Process medicine replacements
    R = get_reactions(data)
    M = set()
    for k, v in R.items():
        for mol in v:
            M = M.union(repl_all(medicine, k, mol))

    return len(M)

data = get_input('input.txt')
part1(data)

### Greedy BFS - perform string replacements greedily

A solution is not guaranteed to be found in this way with a given starting replacement ordering.  Therefore, if a solution is not found, we shuffle the replacement dictionary and try again.  Because of the mathematical properties of the input, this finds a solution quickly.  Topaz, you sly bastard.


In [ ]:
@aoc_timer
def part2(data, output=False):
    # Remove medicine from input data into new variable
    medicine = data.pop()

    # Reactions in reverse
    R = get_reactions(data, True)

    while True:
        start, end = 'e', medicine
        bfs = deque([(end, 0, end)])
        visited = {medicine: 0}
        while bfs:
            mol, steps, route = bfs.popleft()
            for k, v in R.items():
                if k not in mol:
                    continue
                mol, _steps = repl_greedy(mol, k, v)
                steps += _steps
                if mol in visited and visited[mol] <= steps:
                    continue
                visited[mol] = steps
                bfs.append((mol, steps, route + ' =>\n' + mol))
                break  # Always retry from beginning after making successful reductions
        if start in visited:
            break  # Found solution
        if output:
            print(f'NOT FOUND!\nsteps: {steps}, residual: {mol}\nRetrying with shuffled reactions...\n')
        R = shuffle_dict(R)
    if output:
        print(f'route: {route}, steps: {visited[start]}')
    return visited[start]

data = get_input('input.txt')
part2(data, False)

### Full BFS - replacements one at a time

Poor time complexity: https://www.wolframalpha.com/input/?i=3%2C4%2C7%2C10%2C17%2C24%2C41


In [ ]:
# See above link, too slow for actual input
data = get_input('sample.txt')
medicine = data.pop()
medicine = 'HOHOHO'

# Reactions in reverse
R = get_reactions(data, True)

# BFS
start, end = 'e', medicine
bfs = deque([(end, 0, end)])
visited = {medicine: 0}
cnt = 0
while bfs:
    # Progress indicator
    cnt += 1
    if cnt % 10000 == 0: print ('.', end='', flush=True)
    # BFS
    mol, steps, route = bfs.popleft()
    for k, v in R.items():
        if v == start and mol != k:
            continue
        for m in repl_all(mol, k, v):
            if m in visited and visited[m] <= steps + 1:
                continue
            visited[m] = steps + 1
            bfs.append((m, steps + 1, route + ' => ' + m))

if cnt > 10000:
    print('\n')
print(f'route: {route}, steps: {visited[start]}, iters: {cnt}')